In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Sat Dec  5 19:37:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
!pip install git+https://github.com/kernelmachine/allennlp.git@4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
!pip install transformers==2.4.1
!pip install pytorch-transformers==1.2.0
!git clone https://github.com/allenai/dont-stop-pretraining

!pip install adapter-transformers

  Cloning https://github.com/kernelmachine/allennlp.git (to revision 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7) to /tmp/pip-req-build-krnjoh3r
  Running command git clone -q https://github.com/kernelmachine/allennlp.git /tmp/pip-req-build-krnjoh3r
  Running command git checkout -q 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 133kB 16.9MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 245kB 22.7MB/s 
     |████████████████████████████████| 133kB 27.5MB/s 
     |████████████████████████████████| 5.3MB 28.7MB/s 
     |████████████████████████████████| 163kB 55.6MB/s 
     |████████████████████████████████| 266kB 67.9MB/s 
     |████████████████████████████████| 317kB 63.6MB/s

In [4]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [5]:
%cd temp/adapticons/
%mkdir datasets
%cd datasets
%mkdir rct-20k
%cd rct-20k
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/rct-20k/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/rct-20k/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/rct-20k/test.jsonl
%cd ../..

/content/temp/adapticons
/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/rct-20k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.7M  100 34.7M    0     0  9542k      0  0:00:03  0:00:03 --:--:-- 9540k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5959k  100 5959k    0     0  2469k      0  0:00:02  0:00:02 --:--:-- 2468k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5920k  100 5920k    0     0  2674k      0  0:00:02  0:00:02 --:--:-- 2674k
/content/temp/adapticons


In [6]:
%cd modeling

/content/temp/adapticons/modeling


In [7]:
!ls ../datasets/rct-20k

dev.jsonl  test.jsonl  train.jsonl


In [9]:
!ls ../../../gdrive/MyDrive/no_pt/rct

ag	  citation_intent  hyperpartisan_news  rct-20k
chemprot  helpfulness	   imdb		       scierc


In [ ]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/rct-20k \
  --max_seq_length 512 \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 2 \
  --learning_rate 1e-4 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/no_pt/rct \
  --task_name rct \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter_wop \
  --model_name_or_path roberta-base \
  --adapter_config pfeiffer \
  --metric micro \

2020-12-05 19:44:50.593828: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 19:44:52 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 19:44:52 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/no_pt/rct', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir